<a href="https://colab.research.google.com/github/spaziochirale/CorsoPythonML/blob/master/CIFAR_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Convolutional Neural Network (CNN)

In questa lazione mostreremo la costruzione e l'addestramento di una semplice Rete Neurale Convoluzionale.
La rete sarà addestrata per classificare le immagini del dataset  [CIFAR images](https://www.cs.toronto.edu/~kriz/cifar.html). 
L'impiego delle [Keras Sequential API](https://www.tensorflow.org/guide/keras/overview) ci permetterà di raggiungre l'obiettivo con una manciata di linee di codice.


### Importiamo la libreria TensorFlow

In [0]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### Scarichiamo e prepariamo il dataset CIFAR10 

Il dataset CIFAR10 contiene 60.000 immagini a colori relative a 10 categorie di soggetti. Per ogni soggetto sono presenti 6.000 fotografie in bassa risoluzione (32X32 pixel).
Il Dataset è suddiviso in 50.000 immagini di training e 10.000 immagini di test.
Le categorie, o classi, sono mutualmente esclusive, cioè ogni oggetto può appartenere a una sola classe.

In [0]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### Diamo uno sguardo ai nostri dati

Per vedere come sono fatti i nostri dati, disegnamo a video, con l'aiuto della consueta libreria grafica *matplotlib*, le prime 25 immagini del dataset di training, e stampiamo sotto ogni immagine la relativa etichetta.  


In [0]:
class_names = ['aeroplano', 'automobile', 'uccello', 'gatto', 'cervo',
               'cane', 'rana', 'cavallo', 'nave', 'camion']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### Creiamo la parte convoluzionale della rete

Le sei righe di codice, riportate qui sotto, definiscono la parte convoluzionale utilizzando il consueto schema costituito da una sequenza di layer[Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) e [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D).

L'input alla nostra CNN è un tensore con shape (image_height, image_width, color_channels), per il momento ignoriamo il concetto di batch size che il lettore più esperto di reti neurali potrebbe aver considerato. Color_channels si riferisce alla rappresentazione (R,G,B). 
Nel nostro caso, l'input della CNN è  un tensore con shape (32, 32, 3), che è il formato delle immagini CIFAR, cioè matrici 32X32 bit a colori in RGB. Il parametro `input_shape` del nostro primo layer serve ad impostare questa *forma* per l'input.
Il primo parametro del metodo `layers.conv2D` definisce il numero di canali di output dello strato.


In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Possiamo stampare a video un riepilogo della struttura costruita fino a questo punto con la seguente chiamata del metodo `summary`()

In [0]:
model.summary()

Come si può osservare, l'output di ogni strato Conv2D e MaxPooling2D è un tensore 3D con shape (height, width, channels). Man mano che si procede verso gli strati interni le dimensioni width e height tendono a diminuire. Pertanto, il numero di canali di output di ciascun layer Conv2D può essere aumentato (ad esempio, 32 o 64) senza rischiare di sovraccaricare l'elaborazione.

### Aggiungiamo i layer di tipo Dense nella parte finale della rete
Per completare il nostro modello, non ci resta che aggiungere la parte di rete DNN a cui sarà demandata l'operazione di classificazione.
L'output dell'ultimo strato della base convoluzionale è un tensore con shape (4, 4, 64). Gli strati di tipo Dense accettano in input dei vettori, cioè dei tensori 1D, mentre l'output corrente è un tensore 3D. Pertanto, come prima cosa, dobbiamo effettuare l'operazione di flatten dell'output 3D, e successivamente aggiungere uno o più layer Dense. 
Il CIFAR ha 10 classi di output, per cui utilizzeremo un ultimo Dense layer con 10 outputs e una softmax come funzione di attivazione.

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

L'architettura finale del modello è pertanto questa:.

In [0]:
model.summary()

Possiamo osservare che l'output con shape (4, 4, 64) è stato appiattito (flattened) in un vettore di dimensione (1024) prima di essere trasferito allo strato dense di 64 neuroni.

### Compiliamo e addestriamo il modello

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

### Valutiamo l'accuratezza raggiunta dalla rete

In [0]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [0]:
print(test_acc)

La nostra semplice rete CNN ha raggiunto un accuratezza superiore al 70%. Non male per un esercizio risolto in così poco tempo!
